## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Marysville,US,48.0518,-122.1771,60.49,73,90,1.99,overcast clouds
1,1,Ucluelet,CA,48.9329,-125.5528,56.08,90,90,1.41,overcast clouds
2,2,Nanortalik,GL,60.1432,-45.2371,39.36,70,63,6.76,broken clouds
3,3,Ushuaia,AR,-54.8000,-68.3000,29.82,100,75,3.44,broken clouds
4,4,Gobabis,NaN,-22.4500,18.9667,59.79,28,0,11.16,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 

min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.

preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
6,6,Nikolskoye,RU,59.7035,30.7861,75.79,43,5,2.84,clear sky
14,14,Lardos,GR,36.0942,28.0157,77.95,56,0,14.54,clear sky
18,18,Cabedelo,BR,-6.9811,-34.8339,84.87,68,31,17.00,scattered clouds
21,21,Atuona,PF,-9.8000,-139.0333,79.41,81,29,19.06,scattered clouds
22,22,Ambon,ID,-3.6954,128.1814,80.11,79,100,7.43,overcast clouds
23,23,Akyab,MM,20.1500,92.9000,84.97,79,100,9.95,overcast clouds
24,24,Banda Aceh,ID,5.5577,95.3222,77.07,83,100,11.52,light rain
26,26,Kapaa,US,22.0752,-159.3190,75.52,85,90,1.01,light rain
32,32,Arraial Do Cabo,BR,-22.9661,-42.0278,78.76,78,40,4.61,scattered clouds
34,34,Bathsheba,BB,13.2167,-59.5167,88.41,55,40,18.41,scattered clouds


In [5]:
# 4a. Determine if there are any empty rows.

preferred_cities_df.count()

City_ID                278
City                   278
Country                277
Lat                    278
Lng                    278
Max Temp               278
Humidity               278
Cloudiness             278
Wind Speed             278
Current Description    278
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.

preferred_cities_df = preferred_cities_df.dropna()

clean_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()
clean_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng
6,Nikolskoye,RU,75.79,clear sky,59.7035,30.7861
14,Lardos,GR,77.95,clear sky,36.0942,28.0157
18,Cabedelo,BR,84.87,scattered clouds,-6.9811,-34.8339
21,Atuona,PF,79.41,scattered clouds,-9.8000,-139.0333
22,Ambon,ID,80.11,overcast clouds,-3.6954,128.1814
23,Akyab,MM,84.97,overcast clouds,20.1500,92.9000
24,Banda Aceh,ID,77.07,light rain,5.5577,95.3222
26,Kapaa,US,75.52,light rain,22.0752,-159.3190
32,Arraial Do Cabo,BR,78.76,scattered clouds,-22.9661,-42.0278
34,Bathsheba,BB,88.41,scattered clouds,13.2167,-59.5167


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
6,Nikolskoye,RU,75.79,clear sky,59.7035,30.7861,
14,Lardos,GR,77.95,clear sky,36.0942,28.0157,
18,Cabedelo,BR,84.87,scattered clouds,-6.9811,-34.8339,
21,Atuona,PF,79.41,scattered clouds,-9.8000,-139.0333,
22,Ambon,ID,80.11,overcast clouds,-3.6954,128.1814,
23,Akyab,MM,84.97,overcast clouds,20.1500,92.9000,
24,Banda Aceh,ID,77.07,light rain,5.5577,95.3222,
26,Kapaa,US,75.52,light rain,22.0752,-159.3190,
32,Arraial Do Cabo,BR,78.76,scattered clouds,-22.9661,-42.0278,
34,Bathsheba,BB,88.41,scattered clouds,13.2167,-59.5167,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
       hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
       print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.

# inside brackets: hotel name in hotel_df that doesn't equal empty string
clean_hotel_df = hotel_df[hotel_df["Hotel Name"] != '']

In [10]:
# 8a. Create the output File (CSV)
output_data_file = "../Vacation_Search/WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """

"""


info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd> 
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
clean_hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

# Store the DataFrame Row.
#hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [12]:
# 11a. Add a marker layer for each city to the map. 

# 11b. Display the figure

# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=clean_hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))